In [1]:
import numpy as np
import pandas as pd

In [2]:
df_t = pd.read_csv('../final/joinedClimateJuly.csv')
df_avg_electricity = pd.read_csv('../avg_electricity/avg_electricity_july.csv')

In [3]:
df = pd.merge(df_t, df_avg_electricity, left_on=['TOWNNAME', 'VILLNAME'], right_on=['鄉鎮名稱', '村里名稱'],how='left')


In [4]:
result_x = pd.DataFrame()
result_y = pd.DataFrame()

for i in range(11,15):
    year = i
    col_list = []
    rename_dict = {f"人均用電量_20{year}":"人均用電量"}
    x_col = []

    for a in range(24):
        col_list.append(f'T_{year}_{a}')
        x_col.append(f'T_{a}')
        rename_dict[f'T_{year}_{a}'] = f'T_{a}'

    df_temp = df[[f"人均用電量_20{year}"] + col_list]
    df_temp=df_temp.dropna(axis=0)
    df_temp = df_temp.rename(columns=rename_dict)

    df_x_temp = df_temp[x_col]

    df_y_temp = df_temp[["人均用電量"]]
    result_x = pd.concat([result_x, df_x_temp],axis=0)
    result_y = pd.concat([result_y, df_y_temp],axis=0)

In [5]:
taipei_center = ['松山區','信義區','大安區','中山區','中正區']
x = result_x
y = result_y["人均用電量"]

In [6]:
arr_X = x.to_numpy()
arr_y = y.to_numpy()

In [7]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [8]:
lm = LinearRegression()
lm.fit(arr_X, arr_y)

# 印出係數
print(lm.coef_)

# 印出截距
print(lm.intercept_ )

[ 748.11373124 -455.01263392   17.42520102   81.45773677   99.51152784
 -358.5883891    86.82462575   51.65160256   -9.18235495   61.26521346
  -73.68873982   47.36799314    2.71273345   56.8279075  -104.08076549
    1.40683022  -20.58864563   37.21402946 -145.77552218  658.28423911
 -736.43258393   26.03420011 -454.61576441  460.56959899]
-1757.5622556912467


In [9]:
# 模型績效
mse = np.mean((lm.predict(arr_X) - arr_y) ** 2)
r_squared = lm.score(arr_X, arr_y)
adj_r_squared = r_squared - (1 - r_squared) * (arr_X.shape[1] / (arr_X.shape[0] - arr_X.shape[1] - 1))

# 印出模型績效
print(mse)
print(r_squared)
print(adj_r_squared)

32640.431479772305
0.04716252278944866
0.013433762534207902


In [10]:
from sklearn.feature_selection import f_regression

In [11]:
# 印出 p-value
print(f_regression(arr_X, arr_y)[1])

[0.1551521  0.13846859 0.17659563 0.23211446 0.2263613  0.31029177
 0.3923223  0.59598008 0.54542756 0.43695181 0.39676614 0.22453815
 0.53192041 0.59462424 0.37982187 0.81315226 0.23347476 0.66852128
 0.89906292 0.57975562 0.38896749 0.24246876 0.21001567 0.19263529]


In [13]:
import statsmodels.api as sm


In [14]:
#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

d:\Users\63742\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [15]:
#calculate sse
sse = np.sum((model.fittedvalues - y)**2)
print(sse)

#calculate ssr
ssr = np.sum((model.fittedvalues - y.mean())**2)
print(ssr)

#calculate sst
sst = ssr + sse
print(sst)

22946223.33027993
1135767.4384453339
24081990.768725265


In [19]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  人均用電量   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.398
Date:                Wed, 13 Jul 2022   Prob (F-statistic):             0.0983
Time:                        17:38:39   Log-Likelihood:                -4650.8
No. Observations:                 703   AIC:                             9352.
Df Residuals:                     678   BIC:                             9465.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1757.5623   1341.189     -1.310      0.190   -4390.946     875.821
T_0          748.1137    359.805      2.079      0.038      41.647    1454.580
T_1         -455.0126    323.022     -1.409      0.159   -1089.256     179.231
T_2           17.4252    212.382      0.082      0.935    -399.580     434.431
T_3           81.4577    173.403      0.470      0.639    -259.014     421.929
T_4           99.5115    176.294      0.564      0.573    -246.636     445.659
T_5         -358.5884    187.537     -1.912      0.056    -726.812       9.636
T_6           86.8246    118.605      0.732      0.464    -146.053     319.702
T_7           51.6516     77.619      0.665      0.506    -100.750     204.054
T_8           -9.1824     53.753     -0.171      0.864    -114.724      96.359
T_9           61.2652     54.066      1.133      0.258     -44.892     167.422
T_10         -73.6887     37.829     -1.948      0.052    -147.965       0.588
T_11          47.3680     27.715      1.709      0.088      -7.050     101.786
T_12           2.7127     28.978      0.094      0.925     -54.185      59.611
T_13          56.8279     33.617      1.690      0.091      -9.179     122.835
T_14        -104.0808     44.337     -2.347      0.019    -191.136     -17.026
T_15           1.4068     37.035      0.038      0.970     -71.311      74.125
T_16         -20.5886     24.215     -0.850      0.395     -68.134      26.957
T_17          37.2140     47.577      0.782      0.434     -56.201     130.629
T_18        -145.7755    105.967     -1.376      0.169    -353.839      62.288
T_19         658.2842    307.960      2.138      0.033      53.614    1262.955
T_20        -736.4326    476.326     -1.546      0.123   -1671.685     198.820
T_21          26.0342    541.784      0.048      0.962   -1037.742    1089.810
T_22        -454.6158    436.520     -1.041      0.298   -1311.709     402.477
T_23         460.5696    312.562      1.474      0.141    -153.136    1074.275
==============================================================================
Omnibus:                       59.102   Durbin-Watson:                   1.866
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               28.206
Skew:                           0.305   Prob(JB):                     7.50e-07
Kurtosis:                       2.231   Cond. No.                     3.17e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""